## Training of the DeepESD model

In [1]:
DATA_PATH = './data'
FIGURES_PATH = './figures'
MODELS_PATH = './models'

In [7]:
import xarray as xr
from IPython.display import IFrame

import sys; sys.path.append('../')
import deep4downscaling

In [3]:
# Load predictors
predictor_filename = f'{DATA_PATH}/ERA5_NorthAtlanticRegion_1-5dg_full.nc'
predictor = xr.open_dataset(predictor_filename)
print(predictor)

<xarray.Dataset> Size: 1GB
Dimensions:  (lon: 42, lat: 31, time: 16071)
Coordinates:
  * lon      (lon) float64 336B -39.0 -37.5 -36.0 -34.5 ... 18.0 19.5 21.0 22.5
  * lat      (lat) float64 248B 23.5 25.0 26.5 28.0 29.5 ... 64.0 65.5 67.0 68.5
  * time     (time) datetime64[ns] 129kB 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/13)
    t500     (time, lat, lon) float32 84MB ...
    t700     (time, lat, lon) float32 84MB ...
    t850     (time, lat, lon) float32 84MB ...
    q500     (time, lat, lon) float32 84MB ...
    q700     (time, lat, lon) float32 84MB ...
    q850     (time, lat, lon) float32 84MB ...
    ...       ...
    v700     (time, lat, lon) float32 84MB ...
    v850     (time, lat, lon) float32 84MB ...
    u500     (time, lat, lon) float32 84MB ...
    u700     (time, lat, lon) float32 84MB ...
    u850     (time, lat, lon) float32 84MB ...
    msl      (time, lat, lon) float32 84MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-06 07:2

In [11]:
# Visualize predictors
fig_predictor_climatology_path = f'./{FIGURES_PATH}/predictor_climatology.pdf'
deep4downscaling.viz.multiple_map_plot(data=predictor.mean('time'),
                                       output_path=fig_predictor_climatology_path)
IFrame(fig_predictor_climatology_path, width=1000, height=500)

In [33]:
# Load predictand
predictand_filename = f'{DATA_PATH}/pr_AEMET.nc'
predictand = xr.open_dataset(predictand_filename)
print(predictand)

<xarray.Dataset> Size: 10GB
Dimensions:  (lon: 400, lat: 251, time: 25933)
Coordinates:
  * lon      (lon) float64 3kB -13.18 -13.12 -13.07 -13.02 ... 6.675 6.725 6.775
  * lat      (lat) float64 2kB 33.48 33.52 33.57 33.62 ... 45.87 45.92 45.97
  * time     (time) datetime64[ns] 207kB 1951-01-01 1951-01-02 ... 2021-12-31
Data variables:
    pr       (time, lat, lon) float32 10GB ...


In [38]:
# Visualize predictand
day_to_viz = '10-04-2015'
fig_predictand_day_path = f'./{FIGURES_PATH}/predictand_day.pdf'
deep4downscaling.viz.simple_map_plot(data=predictand.sel(time=day_to_viz),
                                     colorbar='hot_r', var_to_plot='pr',
                                     output_path=fig_predictand_day_path)
IFrame(fig_predictand_day_path, width=500, height=500)

In [39]:
# Remove days with nans in the predictor
predictor = trans.remove_days_with_nans(predictor)

There are no observations containing null values


In [40]:
# Align both datasets in time
predictor, predictand = trans.align_datasets(predictor, predictand, 'time')

In [41]:
# Subset into training and test sets
years_train = ('1980', '2010')
years_test = ('2011', '2020')

x_train = predictor.sel(time=slice(*years_train))
y_train = predictand.sel(time=slice(*years_train))

x_test = predictor.sel(time=slice(*years_test))
y_test = predictand.sel(time=slice(*years_test))